In [1]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import math
from sklearn.feature_extraction import image
from sklearn.cluster import spectral_clustering
from moviepy.editor import VideoFileClip

In [2]:
def save(image,name):
    name=""+str(name)+".png"
    cv2.imwrite(name,image)


In [3]:
def grayen(image):
    grayenn = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return grayenn

In [4]:
def darken(image):
    new_image=np.zeros_like(image)
    alpha=0.19
    beta=-35
    new_image=np.clip(np.multiply(alpha,image)+beta,0,255)
    return new_image

In [5]:
def brighten(image):
    new_image=np.zeros_like(image)
    gamma=0.5
    new_image=np.clip(np.multiply(np.power(np.multiply(1/255,image),gamma),255),0,255)
    return new_image

In [6]:
def colour_threshing(image):
    image = np.where(image < 210, 0, 255)

    return image

In [7]:
def perspective(image):
    h,w=image.shape
    pts1 = np.float32([[h/3,0],[h/3,w-1],[h-1,0],[h-1,w-1]])
    pts2 = np.float32([[0,0],[300,0],[0,300],[300,300]])

    M = cv2.getPerspectiveTransform(pts1,pts2)

    dst = cv2.warpPerspective(median_image,M,(300,300))
    return dst

In [8]:
def roi_mask(image):
    mask=np.zeros_like(image)
    height,width = image.shape
    a3 = np.array( [[[0,200],[1500,200],[width-1,800],[width-1,height-1],[0,height-1]]], dtype=np.int32 )
    cv2.fillPoly(mask,a3,255)
    return mask

In [9]:
def white_pixels(image,x,y):
    h=window_h
    w=window_w
    sum1=np.sum(combo[y-h:y,x:x+w],axis=0).reshape((1,w))
    sum2=np.sum(sum1,axis=1)
    return sum2/255

In [10]:
def centroid(rect1x):
    he=rect1x.shape[0]
    wi=rect1x.shape[1]
    sum1x=0
    sum1y=0
    for i in range(he):
        for j in range(wi):
            sum1x=sum1x+j*rect1x[i,j]*(1/255)
            sum1y=sum1y+i*rect1x[i,j]*(1/255)
    sum1x=int(sum1x/window_w)
    sum1y=int(sum1y/window_h)
    return [sum1x,sum1y]

In [11]:
def yellow_black(image):
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    yellow_lo=np.array([25,175,175])
    yellow_hi=np.array([35,255,255])

    mask=cv2.inRange(hsv,yellow_lo,yellow_hi)
    
    img=np.copy(image)
    img[mask>0]=(0,0,0)

    return img

In [12]:
def green_white(image):
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    green_lo=np.array([20,40,40])
    green_hi=np.array([80,255,255])

    mask=cv2.inRange(hsv,green_lo,green_hi)
    img=np.copy(image)
    img[mask>0]=(255,255,255)

    return img

In [13]:
def binary(image):
    img=np.copy(image)
    img=np.where(img!=(255,255,255),(0,0,0),(255,255,255))
    return img

In [14]:
def vid_pipeline(img):
    x=0
    y=0
    width= img.shape[1]
    height=img.shape[0]
    crop_image = img[y:y+(int)(height*(5/6)), x:x+width]

    width= crop_image.shape[1]
    height=crop_image.shape[0]
    
    yellow=yellow_black(crop_image)
    green=green_white(yellow)
    masked=binary(green)
    masked = cv2.medianBlur(masked.astype(np.uint8),9)
    
    
    gray=grayen(crop_image)
    brighten_image=brighten(gray)
    bw_image=colour_threshing(brighten_image)
    median_image = cv2.medianBlur(np.uint8(bw_image.astype(np.float32)),15)
    mask=roi_mask(median_image)
    combo=cv2.bitwise_and(mask,median_image)
    edges = cv2.Canny(np.uint8(combo),175,200)
    lines = cv2.HoughLines(edges,1,5*np.pi/180,50)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center2=cv2.kmeans(lines.astype(np.float32),2,None,criteria,10,cv2.KMEANS_PP_CENTERS)
    line_img=np.zeros_like(edges)
    for i in range(len(center2)):
        rho,theta = center2[i]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 2000*(-b))
        y1 = int(y0 + 2000*(a))
        x2 = int(x0 - 2000*(-b))
        y2 = int(y0 - 2000*(a))
        cv2.line(line_img,(x1,y1),(x2,y2),255,30)
    line_img=cv2.bitwise_and(mask,line_img)    
    line_img=cv2.cvtColor(line_img,cv2.COLOR_GRAY2RGB)
   
    crop_image=np.where(masked==(0,0,0),crop_image*0,crop_image*1)
    crop_image=np.where(line_img==(255,255,255),(255,153,51),crop_image*1)
       
    return crop_image

In [15]:
myclip = VideoFileClip('sample_output.mp4')
output_vid = 'output3.mp4'
clip = myclip.fl_image(vid_pipeline)
clip.write_videofile(output_vid, audio=True)

chunk:   8%|▊         | 102/1324 [00:00<00:01, 952.73it/s, now=None]

Moviepy - Building video output3.mp4.
MoviePy - Writing audio in output3TEMP_MPY_wvf_snd.mp3


t:   0%|          | 0/3598 [00:00<?, ?it/s, now=None]                

MoviePy - Done.
Moviepy - Writing video output3.mp4



t:  94%|█████████▍| 3386/3598 [30:55<01:55,  1.84it/s, now=None]

error: OpenCV(4.5.1) ../modules/core/src/kmeans.cpp:243: error: (-2:Unspecified error) in function 'double cv::kmeans(cv::InputArray, int, cv::InputOutputArray, cv::TermCriteria, int, int, cv::OutputArray)'
> Number of clusters should be more than number of elements (expected: 'N >= K'), where
>     'N' is 1
> must be greater than or equal to
>     'K' is 2


In [22]:
save(vid_pipeline(cv2.imread("frame6.png")),"o6")

In [23]:

cap= cv2.VideoCapture('sample_output.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('f'+str(i)+'.jpg',vid_pipeline(frame))
    i+=1
 
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.1) ../modules/core/src/kmeans.cpp:243: error: (-2:Unspecified error) in function 'double cv::kmeans(cv::InputArray, int, cv::InputOutputArray, cv::TermCriteria, int, int, cv::OutputArray)'
> Number of clusters should be more than number of elements (expected: 'N >= K'), where
>     'N' is 1
> must be greater than or equal to
>     'K' is 2
